<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/sgrepo/BackgroundDataAugumentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# freepik back ground image roomというところから取れる写真がいい

Installation

In [ ]:
!pip install -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install -q diffusers

Download demo data

In [ ]:
!wget https://www.noce.co.jp/client_info/NOCE/itemimage/2310000029344/2310000029344_01.jpg -O /content/chair.jpg

Fix point

In [ ]:
# Please change this line in /usr/local/lib/python3.10/dist-packages/transformers/models/oneformer/image_processing_oneformer.py
"""
# class_queries_logits = outputs.class_queries_logits  # [batch_size, num_queries, num_classes+1]
# masks_queries_logits = outputs.masks_queries_logits  # [batch_size, num_queries, height, width]
class_queries_logits = outputs.transformer_decoder_class_predictions
masks_queries_logits = outputs.transformer_decoder_mask_predictions
"""

Pipeline function

In [1]:
import torch
import os
from PIL import Image
import numpy as np
import cv2
from transformers import OneFormerProcessor, OneFormerModel
from torch import autocast
from diffusers import StableDiffusionImg2ImgPipeline

In [2]:
# load oneformer model
model_id = "shi-labs/oneformer_ade20k_swin_large"
processor = OneFormerProcessor.from_pretrained(model_id)
model = OneFormerModel.from_pretrained(model_id)

# load stable diffusion model
model_id = "SG161222/Realistic_Vision_V5.1_noVAE"
device = "cuda"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/models/oneformer/image_processing_oneformer.py:424: FutureWarning: The `reduce_labels` argument is deprecated and will be removed in v4.27. Please use `do_reduce_labels` instead.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
def retrieve_mask(label_name, bg_name):
  image = Image.open(f"/content/{label_name}.jpg")

  inputs = processor(image, ["semantic"], return_tensors="pt")

  with torch.no_grad():
      outputs = model(**inputs)

  # you can pass them to processor for semantic postprocessing
  predicted_semantic_map = processor.post_process_semantic_segmentation(
      outputs, target_sizes=[image.size[::-1]]
  )[0]

  seg_id = model.config.label2id[label_name]
  label_pred_map = np.where(predicted_semantic_map == seg_id, 255, 0)

  cv2.imwrite(f"/content/{label_name}_mask.jpg", label_pred_map.astype(np.uint8))


def create_alphablend(label_name, bg_name, ratio, x, y):
  fg_img = cv2.imread(f"/content/{label_name}.jpg")
  fh, fw = fg_img.shape[:2]
  bg_img = cv2.imread(f"/content/{bg_name}.jpg")
  bh, bw = bg_img.shape[:2]
  bg_img = cv2.resize(bg_img, (int(bw * ratio), int(bh * ratio)))

  mask = cv2.cvtColor(cv2.imread(f"/content/{label_name}_mask.jpg"), cv2.COLOR_BGR2GRAY)
  mask = cv2.resize(mask, (fw, fh))

  w = min(fg_img.shape[1], bg_img.shape[1] - x)
  h = min(fg_img.shape[0], bg_img.shape[0] - y)

  fg_roi = fg_img[:h, :w]
  bg_roi = bg_img[y:y+h, x:x +w]

  bg_roi[:] = np.where(mask[:h, :w, np.newaxis] == 0, bg_roi, fg_roi)

  rgb = cv2.resize(cv2.cvtColor(bg_img, cv2.COLOR_BGR2RGB), (bw, bh))
  display(Image.fromarray(rgb))
  cv2.imwrite(f"/content/{label_name}_blend.jpg", cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR))


def refine_with_stablediffusion(label_name, bg_name, save_dir, strength=0.2):
  prompt = ""

  init_img = Image.open(f"/content/{label_name}_blend.jpg")

  # パイプラインの実行
  generator = torch.Generator(device).manual_seed(42) # 再現できるようにseedを設定
  with torch.autocast("cuda"):
      image = pipe(prompt, image=init_img, guidance_scale=7.5, strength=strength, generator=generator).images[0]

  image.save(save_dir + f"{label_name}_{bg_name}.jpg")
  display(image)

In [42]:
# Input parameters
label_name = "chair"
bg_name = "0006"
ratio = 4.5
x, y = 800, 680
save_dir = "/content/bgimages/"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
retrieve_mask(label_name, bg_name)
create_alphablend(label_name, bg_name, ratio, x, y)

In [ ]:
refine_with_stablediffusion(label_name, bg_name, save_dir, strength=0.4)

Finetuning

In [ ]:
%cd /content
!git clone https://github.com/huggingface/diffusers.git

In [46]:
import os
os.makedirs("/content/normal/chair", exist_ok=True)

In [50]:
from accelerate.utils import write_basic_config
write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [1]:
%cd /content/diffusers/examples/research_projects/multi_subject_dreambooth
import json

# here we are using parameters for prior-preservation and validation as well.
concepts_list = [
    {
        "instance_prompt":      "drawing of a <new1> chair",
        "class_prompt":         "drawing of a chair",
        "instance_data_dir":    "/content/bgimages/",
        "class_data_dir":       "/content/normal/chair",
    },
]

with open("concepts_list.json", "w") as f:
    json.dump(concepts_list, f, indent=4)

/content/diffusers/examples/research_projects/multi_subject_dreambooth


In [3]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00


In [ ]:
!accelerate launch train_multi_subject_dreambooth.py \
  --pretrained_model_name_or_path="SG161222/Realistic_Vision_V5.1_noVAE"  \
  --output_dir="testsinglecheckpoints" \
  --train_text_encoder \
  --with_prior_preservation \
  --prior_loss_weight=1.0 \
  --concepts_list="concepts_list.json" \
  --num_class_images=200 \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=1500 \
  --gradient_accumulation_steps=1 --gradient_checkpointing \
  --use_8bit_adam \
  --set_grads_to_none

In [6]:
# inference
%cd /content/diffusers/examples/research_projects/multi_subject_dreambooth
from diffusers import StableDiffusionPipeline
import torch

model_id = "testsinglecheckpoints"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

/content/diffusers/examples/research_projects/multi_subject_dreambooth


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
prompt = "a <new1> chair with table"
negative_prompt = "worst quality, ugly, deformed, tiling, poor quality, picture frame, poorly drawn, mutation, duplicate"
generator = torch.Generator(device="cuda").manual_seed(-1)
images = pipe(
    prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=100,
    guidance_scale=7.5,
    num_images_per_prompt=4
).images


for img in images:
  display(img)

Tips

In [10]:
# print(outputs.transformer_decoder_class_predictions.shape)
# print(outputs.transformer_decoder_mask_predictions.shape)

torch.Size([1, 250, 151])
torch.Size([1, 250, 160, 160])
